# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Pathing casv and making a df
csv_path = "../output_data/cities.csv"
df = pd.read_csv(csv_path)
df

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Max Temperature,Humidity,Cloud Coverage,Wind Speed
0,0,bijie,CN,27.3125,105.2942,1643949306,35.10,90,100,5.88
1,1,tasiilaq,GL,65.6145,-37.6368,1643949306,12.36,57,86,5.37
2,2,bredasdorp,ZA,-34.5322,20.0403,1643949307,59.29,77,100,4.61
3,3,seguela,CI,7.9611,-6.6731,1643949307,65.12,16,1,0.56
4,4,arraial do cabo,BR,-22.9661,-42.0278,1643949307,75.42,91,100,6.26
...,...,...,...,...,...,...,...,...,...,...
579,579,mabaruma,GY,8.2000,-59.7833,1643949468,71.49,95,43,2.42
580,580,taunggyi,MM,20.7833,97.0333,1643949469,71.65,22,2,4.59
581,581,ndola,ZM,-12.9587,28.6366,1643949469,62.35,98,97,4.54
582,582,komatipoort,ZA,-25.4332,31.9548,1643949470,79.05,83,55,6.55


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
#Configure gmaps
gmaps.configure(g_key)

#Use the Lat and Lng as locations and Humidity as the weight
loc = df[["Latitude" , "Longitude"]]
humid = df["Humidity"].astype(int)
so_humid = df["Humidity"].max()

#Add Heatmap layer to map
heat = gmaps.heatmap_layer(loc , weights = humid , point_radius = 3 , max_intensity = so_humid , dissipating = False)
figure = gmaps.figure()

#Display the figure
figure.add_layer(heat)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
#Drop any rows will null values.
clean_df = df.dropna()

#Narrow down the cities to fit weather conditions
#A max temperature lower than 80 degrees but higher than 70
#Wind speed less than 10 mph
#Zero cloudiness
clean_df = clean_df[(clean_df["Wind Speed"] <= 10)]
clean_df = clean_df[(clean_df["Cloud Coverage"] == 0)]
clean_df = clean_df[(clean_df["Max Temperature"] <= 80)]
clean_df = clean_df[(clean_df["Max Temperature"] >= 70)]
clean_df = clean_df[(clean_df["Humidity"] <= 87)]
clean_df.drop(clean_df.filter(regex="Unname"),axis=1, inplace=True)
clean_df

,City,Country,Latitude,Longitude,Date,Max Temperature,Humidity,Cloud Coverage,Wind Speed
63,sur,OM,22.5667,59.5289,1643949102,72.16,40,0,3.71
110,kapaa,US,22.0752,-159.3190,1643949285,75.18,71,0,5.75
186,keti bandar,PK,24.1447,67.4497,1643949357,73.33,39,0,6.04
253,zirandaro,MX,18.4500,-100.9833,1643949376,73.58,47,0,4.00
364,makakilo city,US,21.3469,-158.0858,1643949408,75.52,56,0,9.22
400,celestun,MX,20.8667,-90.4000,1643949419,77.94,63,0,9.26
420,puerto escondido,MX,15.8500,-97.0667,1643949424,74.23,78,0,5.75
566,acapulco,MX,16.8634,-99.8901,1643949395,76.82,78,0,4.61
578,pochutla,MX,15.7432,-96.4661,1643949468,72.82,71,0,4.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = clean_df
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Latitude,Longitude,Date,Max Temperature,Humidity,Cloud Coverage,Wind Speed,Hotel Name
63,sur,OM,22.5667,59.5289,1643949102,72.16,40,0,3.71,NaN
110,kapaa,US,22.0752,-159.3190,1643949285,75.18,71,0,5.75,NaN
186,keti bandar,PK,24.1447,67.4497,1643949357,73.33,39,0,6.04,NaN
253,zirandaro,MX,18.4500,-100.9833,1643949376,73.58,47,0,4.00,NaN
364,makakilo city,US,21.3469,-158.0858,1643949408,75.52,56,0,9.22,NaN
400,celestun,MX,20.8667,-90.4000,1643949419,77.94,63,0,9.26,NaN
420,puerto escondido,MX,15.8500,-97.0667,1643949424,74.23,78,0,5.75,NaN
566,acapulco,MX,16.8634,-99.8901,1643949395,76.82,78,0,4.61,NaN
578,pochutla,MX,15.7432,-96.4661,1643949468,72.82,71,0,4.05,NaN


In [42]:
#Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap

#Create a container to hold outputs to later append
hotel = []

#Create a for loop to iterate through our locations to find hotels
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params = {"radius": 5000,
              "types": "lodging",
              "location" : f"{lat} , {lng}",
              "key": g_key}
    
    #This is the api url and json request to actually get the results
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params).json()
    results = response['results']
    
    #This try and except will make sure I am getting actual hotels into my df
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        hotels.append(np.nan)
        
hotel_df

,City,Country,Latitude,Longitude,Date,Max Temperature,Humidity,Cloud Coverage,Wind Speed,Hotel Name
63,sur,OM,22.5667,59.5289,1643949102,72.16,40,0,3.71,Sur Plaza Hotel
110,kapaa,US,22.0752,-159.3190,1643949285,75.18,71,0,5.75,Sheraton Kauai Coconut Beach Resort
186,keti bandar,PK,24.1447,67.4497,1643949357,73.33,39,0,6.04,Haji laloo memon house
253,zirandaro,MX,18.4500,-100.9833,1643949376,73.58,47,0,4.00,Hotel Pineda
364,makakilo city,US,21.3469,-158.0858,1643949408,75.52,56,0,9.22,Marriott's Ko Olina Beach Club
400,celestun,MX,20.8667,-90.4000,1643949419,77.94,63,0,9.26,Posada Lilia
420,puerto escondido,MX,15.8500,-97.0667,1643949424,74.23,78,0,5.75,Aldea del Bazar Hotel and Spa
566,acapulco,MX,16.8634,-99.8901,1643949395,76.82,78,0,4.61,HS Hotsson Smart Acapulco
578,pochutla,MX,15.7432,-96.4661,1643949468,72.82,71,0,4.05,Hotel Posada San Jose


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [44]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(hotel_df[["Latitude" , "Longitude"]])
figure.add_layer(marker)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))